# Lifetimes, Annotations & Internal mutablility

## Types of &str

* **String literals** - they last for the whole program (they are written directly into the binary)
  * type - `&'static str`

In [6]:
let name: &'static str = "John Smith";

* **Borrowed `str`** - borrowed reference to `String` object (without `'static` lifetime)
  * conversion from `String` to `&str` is allowed thanks to a `Deref` trait

In [ ]:
fn main() {
    let description: String = "And now something completly different...".to_string();
    let borrowed_description: &str = &description;
}

main();